# Section 2

## Efstathios Galanakis 03112172
## Ioannis Tzanettis 03112506
## Team Number: 83



Datasheet Name: SECOM
Number of features: 1567
Number of classes: 591


# Insert Data

First we read data from file and we separate train data from test data.


In [58]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

perc = 0.3
k = 5

features = []
labels = []

labels_path = './big/secom_labels.data'
datasheet_labels = pd.read_csv(labels_path, header=None, delim_whitespace=True)
mapping={-1:0}
labels = datasheet_labels.replace(mapping).values[:,0].flatten()
labels = np.float64(labels)
print labels

[ 0.  0.  1. ...,  0.  0.  0.]


After reading we have to replace missing values

In [59]:
from sklearn.preprocessing import Imputer

imp = Imputer(missing_values='NaN', strategy='mean', axis=0)

features_path = './big/secom.data'
datasheet_features = pd.read_csv(features_path, header=None, delim_whitespace=True)
features = imp.fit_transform(datasheet_features)
print features

[[  3.03093000e+03   2.56400000e+03   2.18773330e+03 ...,   1.64749042e-02
    5.28333333e-03   9.96700663e+01]
 [  3.09578000e+03   2.46514000e+03   2.23042220e+03 ...,   2.01000000e-02
    6.00000000e-03   2.08204500e+02]
 [  2.93261000e+03   2.55994000e+03   2.18641110e+03 ...,   4.84000000e-02
    1.48000000e-02   8.28602000e+01]
 ..., 
 [  2.97881000e+03   2.37978000e+03   2.20630000e+03 ...,   8.60000000e-03
    2.50000000e-03   4.35231000e+01]
 [  2.89492000e+03   2.53201000e+03   2.17703330e+03 ...,   2.45000000e-02
    7.50000000e-03   9.34941000e+01]
 [  2.94492000e+03   2.45076000e+03   2.19544440e+03 ...,   1.62000000e-02
    4.50000000e-03   1.37784400e+02]]


After we replace the missing value, it's time to seperate train and test data

In [60]:
x_train, x_test, y_train, y_test = train_test_split(features,labels,test_size=perc)
y_train = y_train.ravel()
print  features.shape, labels.shape
print x_train.shape, x_test.shape, y_train.shape, y_test.shape


(1567, 590) (1567,)
(1096, 590) (471, 590) (1096,) (471,)


# Hyperparameter Estimation
## Pipeline


In [64]:
from sklearn.model_selection import train_test_split

percentage = 0.30
train_data, test_data, train_labels, test_labels = train_test_split(features, labels, test_size=percentage, random_state=20176)
print features

[[  3.03093000e+03   2.56400000e+03   2.18773330e+03 ...,   1.64749042e-02
    5.28333333e-03   9.96700663e+01]
 [  3.09578000e+03   2.46514000e+03   2.23042220e+03 ...,   2.01000000e-02
    6.00000000e-03   2.08204500e+02]
 [  2.93261000e+03   2.55994000e+03   2.18641110e+03 ...,   4.84000000e-02
    1.48000000e-02   8.28602000e+01]
 ..., 
 [  2.97881000e+03   2.37978000e+03   2.20630000e+03 ...,   8.60000000e-03
    2.50000000e-03   4.35231000e+01]
 [  2.89492000e+03   2.53201000e+03   2.17703330e+03 ...,   2.45000000e-02
    7.50000000e-03   9.34941000e+01]
 [  2.94492000e+03   2.45076000e+03   2.19544440e+03 ...,   1.62000000e-02
    4.50000000e-03   1.37784400e+02]]


In [73]:
from imblearn.pipeline import Pipeline
from sklearn import neighbors
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler 
from imblearn.over_sampling import RandomOverSampler
from sklearn.decomposition import PCA

selector = VarianceThreshold()
scaler = StandardScaler()
sampler = RandomOverSampler()
pca = PCA()
classifier = neighbors.KNeighborsClassifier(n_jobs=-1) 
pipe = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('sampler', sampler), ('pca', pca), ('kNN', classifier)], memory = 'tmp')

In [74]:
print type(train_data),type(train_labels)
print train_data.shape,train_labels.shape
pipe.fit(train_data,train_labels)
pred = pipe.predict(test_data)
print classification_report(test_labels, pred)

<type 'numpy.ndarray'> <type 'numpy.ndarray'>
(1096, 590) (1096,)
             precision    recall  f1-score   support

        0.0       0.94      0.83      0.88       437
        1.0       0.12      0.29      0.17        34

avg / total       0.88      0.79      0.83       471



## Grid Search


In [76]:
from sklearn.model_selection import GridSearchCV
threshold = [0, 0.01, 0.005]
comps = [20, 30, 40, 50, 60]
k = [1, 5, 10, 20, 30, 40]
estimator = GridSearchCV(pipe, dict(selector__threshold=threshold, pca__components=comps, kNN__n_neighbors=k), scoring='f1_macro', n_jobs=-1)

In [77]:
estimator.fit(train_data, train_labels)
pred = estimator.predict(train_data)
print classification_report(train_labels, pred)

             precision    recall  f1-score   support

        0.0       0.97      0.90      0.94      1026
        1.0       0.30      0.64      0.41        70

avg / total       0.93      0.88      0.90      1096



# Result Analysis
## Best Estimator


In [78]:
print estimator.best_estimator_
print estimator.best_params_

Pipeline(memory='tmp',
     steps=[('selector', VarianceThreshold(threshold=0.005)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('sampler', RandomOverSampler(random_state=None, ratio='auto')), ('pca', PCA(copy=True, iterated_power='auto', n_components=30, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('kNN', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=30, p=2,
           weights='uniform'))])
{'selector__threshold': 0.005, 'pca__n_components': 30, 'kNN__n_neighbors': 30}


## Results Visualization
